In [ ]:
import math
from collections import defaultdict
import json
import csv
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import tweepy
# We must use the API to determine whether the tweets are protected
from tweepy import TweepError

In [ ]:
consumer_key = "PQSsqSxGft69D53XGknXya8cL"
consumer_secret = "dlh6EFLADanVNvdl9Q9Lz9Iv3yQQyHY9vxKTS9BN2skflGd45d"
access_token = "1187382763432771584-rwZbZc9lcOcaVdgxcIgqqCPqoQMo3I"
access_token_secret = "nvnIFiZD9L1u6tZwntl24BtsLhKmkBntJ4dpko82fYKg8"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
mmdUserHistories = defaultdict(list)
tweetAuthors = defaultdict(str)
mmdTweetsToText = defaultdict(str)
allTweetsToText = defaultdict(str)

# Read in the JSON data to get the text of each tweet and build user histories
#TODO: Change the filepath here
with open("/Users/chloelarkin/Desktop/hurricane_harvey_final_data.json") as fin:
    for line in fin:
        json_data = json.loads(line)
        user = json_data['user']['id'] # Get the user ID
        tweet_id = json_data['id']
        tweet_text = json_data['text']  # Get the text of the tweet
        tweetAuthors[tweet_id] = user
        mmdTweetsToText[tweet_id] = tweet_text
        allTweetsToText[tweet_id] = tweet_text
        mmdUserHistories[user].append(tweet_id)
        # try: #TODO: This is where we can filter out private tweets
        #     u = api.get_user(user) # Check if the user's tweets are protected. If they are public, then append to our dataset
        #     if not u.protected:
        #         tweet_id = json_data['id']
        #         userTweetDict[tweet_id] = user
        #         tweet_text = json_data['text'] # Get the text of the tweet
        #         originalTweetText[user].append(tweet_text)
        #         originalTweetIDs[user].append(tweet_id)
        # except TweepError as e:
        #     if e.args[0][0]['code'] == 63: # The error code for suspended users. Skip these people
        #         continue

In [ ]:
# Next, build up a history for each user from the extra tweet corpus
extraTweets = defaultdict(list)
with open("/Users/chloelarkin/Desktop/iraq_earthquake_extras.json") as fin:
    for line in fin:
        json_data = json.loads(line)
        user = json_data['user']['id'] # Get the user ID
        if user in mmdUserHistories.keys(): # Check whether the user ID matches someone in the CrisisMMD group
            tweet_id = json_data['id']
            if tweet_id not in mmdUserHistories[user]:  # Check to make sure you're not duplicating a tweet
                mmdUserHistories[user].append(tweet_id) # Add the current tweet to mmdUserHistories
                tweet_text = json_data['text']  # Get the text of the tweet
                tweetAuthors[tweet_id] = user
                allTweetsToText[tweet_id] = tweet_text

In [ ]:
# For each tweet, store list of full user history tweet IDs
fullHistoryPerTweet = defaultdict(list)
for user in mmdUserHistories:
    for tweet in mmdUserHistories[user].copy():
        fullHistoryPerTweet[tweet] = mmdUserHistories[user].copy()

# For each tweet, make a document out of full history
fullHistoryDocs = defaultdict(str)
for tweet in fullHistoryPerTweet:
    fullHistoryDocs[tweet] = ' '.join(allTweetsToText[t] for t in fullHistoryPerTweet[tweet])

In [ ]:
# Sorted list of authors for each tweet in the full corpus
authors = [tweetAuthors[v] for v in sorted(tweetAuthors.keys())]

# Sorted list of tweet strings for each tweet in the full corpus
allTweetsText = [allTweetsToText[v] for v in sorted(allTweetsToText.keys())]

# Sorted list of tweet strings for JUST the labeled set
mmdTweetsText = [mmdTweetsToText[v] for v in sorted(mmdTweetsToText.keys())]

# Sorted list of user histories for JUST the labeled set
mmdTweetsHistories = [fullHistoryDocs[v] for v in sorted(fullHistoryDocs.keys()) if v in mmdTweetsToText.keys()]

In [ ]:
# Establish TFIDF vectorizer
vectorizer = TfidfVectorizer()

# Fit the vectorizer on ALL LABELED AND UNLABELED TWEETS = the corpus
vectorizer.fit(allTweetsText)

# Get TFIDF vectors for each labeled tweet
tweetVecs = vectorizer.transform(mmdTweetsText)

# Get TFIDF vectors for the user histories of every labeled tweet
userHistoryVecs = vectorizer.transform(mmdTweetsHistories)

# Validation check of number of tweets:
print("There were", len(mmdTweetsToText), "original crisisMMD tweets, and there are", len(allTweetsToText),
      "tweets including the extras we extracted.")

# Validation check of relative dimensions of both matrices:
print("Dimensions of labeled tweet vectors:", tweetVecs.get_shape(),
      "and dimensions of user histories for each labeled tweet:", userHistoryVecs.get_shape())

In [ ]:
# Run SVD dimension reduction on both matrices (separately):
# SVD for individual labeled tweets:
individualSVD = TruncatedSVD(n_components=200, n_iter=7, random_state=42)
tweetFeatureArr = individualSVD.fit_transform(tweetVecs)
tweetFeatureArr = np.array(tweetFeatureArr)
print("Shape of individual tweet feature np array:", tweetFeatureArr.shape)

# SVD for user histories:
historySVD = TruncatedSVD(n_components=200, n_iter=7, random_state=42)
histFeatureArr = historySVD.fit_transform(userHistoryVecs)
histFeatureArr = np.array(histFeatureArr)
print("Shape of history feature np array:", histFeatureArr.shape)

# At the end we have tweetFeatureArr and histFeatureArr each of which store the results from SVD.

In [ ]:
# PART TWO: Get the classification numpy array

# Establish dicts of annotated scores for each tweet ID
text_informative = defaultdict(str) #4000 scores -- this is the binary informative / not informative score
text_humanitarian = defaultdict(str) # 4000 scores -- finer-grain classification

# Read in the classifications of each tweet
with open("/Users/chloelarkin/Desktop/iraq_iran_earthquake_final_data.tsv") as fin:
    reader = csv.DictReader(fin, dialect='excel-tab')
    for line in reader:
        text_informative[line['tweet_id']]=line['text_info']
        text_humanitarian[line['tweet_id']]=line['text_human']

# Retrieve scores for each tweet in the dataset
informative_scores = defaultdict(str)
humanitarian_scores = defaultdict(str)
for id in mmdTweetsToText.keys():
    informative_scores[id] = text_informative["\""+ str(id) + "\""]
for id in mmdTweetsToText.keys():
    humanitarian_scores[id] = text_humanitarian["\""+ str(id) + "\""]

informativeScoreArr = [informative_scores[v] for v in sorted(informative_scores.keys())]
informativeScoreArr = np.array(informativeScoreArr)
print("Informativeness classification np array shape:", informativeScoreArr.shape)

humanitarianScoreArr = [humanitarian_scores[v] for v in sorted(humanitarian_scores.keys())]
humanitarianScoreArr = np.array(humanitarianScoreArr)
print("Humanitarian classification np array shape:", humanitarianScoreArr.shape)

# Sanity check to make sure there is one score for each tweet in the CrisisMMD set (should be 499)
print("There are", len(mmdTweetsToText), "MMD tweets and", len(informative_scores),
      "informative scores and", len(humanitarian_scores), "humanitarian scores.")